In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Import modules
from __future__ import print_function
import tensorflow as tf
import numpy as np
from ecbm4040.cifar_utils import load_data
from matplotlib import pyplot as plt

In [2]:
from ecbm4040.layer_funcs import conv2d_forward

# Set test parameters.
x_shape = (2, 4, 4, 3)
w_shape = (3, 4, 4, 3)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)
pad = 1
stride = 2
your_feedforward = conv2d_forward(x, w, b, pad, stride)
print(your_feedforward)

[[[[-0.04138903  0.11338093  0.2681509 ]
   [-0.09391829  0.14671108  0.38734045]]

  [[-0.31590063  0.18230696  0.68051454]
   [-0.41135959  0.1727074   0.75677438]]]


 [[[-0.79122782  0.7372926   2.26581303]
   [-0.92961649  0.68476334  2.29914317]]

  [[-1.40917703  0.46278101  2.33473905]
   [-1.5904954   0.36732205  2.32513949]]]]


In [6]:
from ecbm4040.layer_funcs import conv2d_backward
# Set test parameters. Please don't change it.
np.random.seed(123)
d_top = np.random.normal(size=your_feedforward.shape)
your_dw, your_db = conv2d_backward(d_top, x, w, b, pad, stride)
print(your_dw)
print('*'*50)
print(your_db)

[-2.33352069  0.32832703  6.76164407]
**************************************************
[[[[ 0.44864302  0.45258402  0.45652502]
   [ 0.79929539  0.79425139  0.78920739]
   [ 0.78416338  0.77911938  0.77407538]
   [ 0.28491936  0.27593436  0.26694936]]

  [[ 0.52155246  0.51323675  0.50492104]
   [ 1.13916531  1.12442728  1.10968926]
   [ 1.09495123  1.08021321  1.06547518]
   [ 0.6040261   0.59760379  0.59118148]]

  [[ 0.42176392  0.41344821  0.4051325 ]
   [ 0.962309    0.94757098  0.93283295]
   [ 0.91809493  0.9033569   0.88861888]
   [ 0.52695834  0.52053603  0.51411371]]

  [[-0.26854363 -0.28080034 -0.29305705]
   [ 0.1677414   0.15804737  0.14835335]
   [ 0.13865933  0.12896531  0.11927128]
   [ 0.48843129  0.49099398  0.49355667]]]


 [[[-0.2752069  -0.28540513 -0.29560336]
   [-0.18769171 -0.1981598  -0.20862788]
   [-0.21909596 -0.22956404 -0.24003212]
   [ 0.11649078  0.11622093  0.11595108]]

  [[ 0.13926008  0.13933972  0.13941936]
   [ 0.02483815  0.02691179  0.0289854

## TensorFlow CNN

In this part we will construct the CNN in TensorFlow. To be more specific, we are going to implement a CNN similar to the LeNet structure.

Tensorflow offers many useful resources and functions which help developers build the net in a high-level fashion, such as functions in the `layer` module. However, we will build the network by ourself for this homework for better understanding. By utilizing functions in `tf.nn` that exist for Neural Network structuring and training, we can build out our own layers and network modules rather quickly.

Also, we will introduce a visualization tool called Tensorboard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data that pass through it.

Resources and References: <br>
* [TensorBoard: Visualizing Learning](https://www.tensorflow.org/get_started/summaries_and_tensorboard)<br>
* [Convolutional Neural Networks (LeNet) - DeepLearning 0.1 documentation](http://deeplearning.net/tutorial/lenet.html)<br>
* [LeNet-5, convolutional neural networks](http://yann.lecun.com/exdb/lenet/)

In [2]:
# Load the raw CIFAR-10 data.
X_train, y_train = load_data(mode='train')

# Data organizations:
# Train data: 49000 samples from original train set: 1~49000
# Validation data: 1000 samples from original train set: 49000~50000
num_training = 49000
num_validation = 1000

X_val = X_train[-num_validation:, :]
y_val = y_train[-num_validation:]

X_train = X_train[:num_training, :]
y_train = y_train[:num_training]

# Preprocessing: subtract the mean value across every dimension for training data, and reshape it to be RGB size
mean_image = np.mean(X_train, axis=0)
X_train = X_train.astype(np.float32) - mean_image.astype(np.float32)
X_val = X_val.astype(np.float32) - mean_image

X_train = X_train.reshape([-1,32,32,3])/255
X_val = X_val.reshape([-1,32,32,3])/255

print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)


### CNN model example

In [8]:
from ecbm4040.neuralnets.cnn_sample import training
tf.reset_default_graph()
training(X_train, y_train, X_val, y_val, 
         conv_featmap=[6],
         fc_units=[84],
         conv_kernel_size=[5],
         pooling_size=[2],
         l2_norm=0.01,
         seed=235,
         learning_rate=1e-2,
         epoch=20,
         batch_size=245,
         verbose=False,
         pre_trained_model=None)

Building my LeNet. Parameters: 
conv_featmap=[6]
fc_units=[84]
conv_kernel_size=[5]
pooling_size=[2]
l2_norm=0.01
seed=235
learning_rate=0.01
number of batches for training: 200
epoch 1 
Best validation accuracy! iteration:100 accuracy: 16.5%
Best validation accuracy! iteration:200 accuracy: 18.400000000000006%
epoch 2 
Best validation accuracy! iteration:300 accuracy: 18.5%
Best validation accuracy! iteration:400 accuracy: 20.0%
epoch 3 
Best validation accuracy! iteration:500 accuracy: 21.599999999999994%
Best validation accuracy! iteration:600 accuracy: 21.799999999999997%
epoch 4 
Best validation accuracy! iteration:700 accuracy: 22.5%
Best validation accuracy! iteration:800 accuracy: 23.700000000000003%
epoch 5 
Best validation accuracy! iteration:900 accuracy: 24.200000000000003%
Best validation accuracy! iteration:1000 accuracy: 25.299999999999997%
epoch 6 
Best validation accuracy! iteration:1100 accuracy: 25.700000000000003%
Best validation accuracy! iteration:1200 accuracy: 2

### Show the model structure graph

In [4]:
# show the graph
from ecbm4040.neuralnets.cnn_jupyter_tensorboard import show_graph 
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model/lenet_1509306364.meta')
    graph = tf.get_default_graph()
    show_graph(graph)

### Custom CNN model

In [3]:
from ecbm4040.neuralnets.cnn import my_training
tf.reset_default_graph()

my_training(X_train, y_train, X_val, y_val, 
         conv_featmap=[32,64,128],
         fc_units=[256],
         conv_kernel_size=[5,5,5],
         dropout_rate=[0.5, 0.75, 1.0],
         pooling_size=[2,2,2],
         l2_norm=0.01,
         seed=235,
         learning_rate=1e-2,
         epoch=150,
         batch_size=2000,
         verbose=False,
         pre_trained_model=None)

Building my LeNet. Parameters: 
conv_featmap=[32, 64, 128]
fc_units=[256]
conv_kernel_size=[5, 5, 5]
pooling_size=[2, 2, 2]
l2_norm=0.01
seed=235
learning_rate=0.01
number of batches for training: 24
epoch 1 
epoch 2 
epoch 3 
epoch 4 
epoch 5 
Best validation accuracy! iteration:100 accuracy: 43.4%
epoch 6 
epoch 7 
epoch 8 
epoch 9 
Best validation accuracy! iteration:200 accuracy: 50.9%
epoch 10 
epoch 11 
epoch 12 
epoch 13 
Best validation accuracy! iteration:300 accuracy: 54.6%
epoch 14 
epoch 15 
epoch 16 
epoch 17 
Best validation accuracy! iteration:400 accuracy: 58.9%
epoch 18 
epoch 19 
epoch 20 
epoch 21 
Best validation accuracy! iteration:500 accuracy: 60.2%
epoch 22 
epoch 23 
epoch 24 
epoch 25 
Best validation accuracy! iteration:600 accuracy: 61.0%
epoch 26 
epoch 27 
epoch 28 
epoch 29 
epoch 30 
Best validation accuracy! iteration:700 accuracy: 62.1%
epoch 31 
epoch 32 
epoch 33 
epoch 34 
epoch 35 
epoch 36 
epoch 37 
epoch 38 
epoch 39 
epoch 40 
epoch 41 
epoch 4

</span>
![Tensorboard_2](./ecbm4040/notebook_images/cnn.PNG)

In [5]:
# show the graph
from ecbm4040.neuralnets.cnn_jupyter_tensorboard import show_graph 
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model/lenet_1509803802.meta')
    graph = tf.get_default_graph()
    show_graph(graph)